# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [113]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict

# Import API key
from api_keys import geoapify_key
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [114]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bejar,40.3864,-5.7634,8.10,90,81,1.09,ES,1670384534
1,1,chokurdakh,70.6333,147.9167,-20.87,97,100,1.13,RU,1670384535
2,2,barrow,71.2906,-156.7887,-8.99,79,100,9.26,US,1670384535
3,3,albany,42.6001,-73.9662,11.43,93,100,3.13,US,1670384249
4,4,kapaa,22.0752,-159.3190,27.18,86,100,5.14,US,1670384536


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [115]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
    )

# Display the map
hvplot.save(map_plot_1, "../output_data/map_plot_1.html")
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [116]:
# Narrow down cities that fit criteria and drop any results with null values
my_ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] >= 15) 
                                & (city_data_df["Max Temp"] <= 24)
                                & (city_data_df["Humidity"] <= 85)
                                & (city_data_df.Cloudiness <= 60), :]

# Drop any rows with null values
my_ideal_weather_df = my_ideal_weather_df.dropna(how='any')

# Display sample data
# my_ideal_weather_df = my_ideal_weather_df.reset_index(drop=False)
my_ideal_weather_df


,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,cape town,-33.9258,18.4232,15.62,82,0,1.03,ZA,1670384079
26,26,matamoros,25.5333,-103.2500,22.06,43,0,0.00,MX,1670384544
35,35,sao filipe,14.8961,-24.4956,22.88,76,16,2.06,CV,1670384548
38,38,nishihara,26.1842,127.7558,21.81,64,20,7.72,JP,1670384549
40,40,mar del plata,-38.0023,-57.5575,19.01,78,40,5.66,AR,1670383926
...,...,...,...,...,...,...,...,...,...,...
485,485,puerto madryn,-42.7692,-65.0385,16.24,61,4,6.81,AR,1670384719
504,504,porbandar,21.6422,69.6093,18.99,52,2,0.00,IN,1670384695
511,511,yol,32.1667,76.2000,17.53,22,32,0.89,IN,1670384729
519,519,san luis,-33.2950,-66.3356,23.18,51,1,5.73,AR,1670384733


### Step 3: Create a new DataFrame called `hotel_df`.

In [117]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_ideal_weather_df[['City','Country', 'Lat', 'Lon', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df
                            

,City,Country,Lat,Lon,Humidity,Hotel Name
17,cape town,ZA,-33.9258,18.4232,82,
26,matamoros,MX,25.5333,-103.2500,43,
35,sao filipe,CV,14.8961,-24.4956,76,
38,nishihara,JP,26.1842,127.7558,64,
40,mar del plata,AR,-38.0023,-57.5575,78,
...,...,...,...,...,...,...
485,puerto madryn,AR,-42.7692,-65.0385,61,
504,porbandar,IN,21.6422,69.6093,52,
511,yol,IN,32.1667,76.2000,22,
519,san luis,AR,-33.2950,-66.3356,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [118]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 100,
    "filter": "",
    "bias": "",
    "apiKey": geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lon"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = (f"circle:{longitude},{latitude},{radius}")
    params["bias"] = (f"proximity:{longitude},{latitude}")
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cape town - nearest hotel: Townhouse Hotel
matamoros - nearest hotel: No hotel found
sao filipe - nearest hotel: Bela Vista
nishihara - nearest hotel: ユインチホテル南城
mar del plata - nearest hotel: Nuevo Ostende
victoria - nearest hotel: 四季酒店 Four Seasons Hotel
bluff - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
kruisfontein - nearest hotel: No hotel found
hobart - nearest hotel: The Old Woolstore Apartment Motel
jaisalmer - nearest hotel: Jeet Mahal
tambura - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
carnarvon - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
namibe - nearest hotel: Hotel Chik Chik Namibe
cidreira - nearest hotel: Hotel Castelo
kathor - nearest hotel: Hotel girnar
san carlos park - nearest hotel: Courtyard by Marriott Fort Myers at I-75 and Gulf Coast Town Center
haimen - nearest hotel: No hotel found
leo - nearest hotel: Auberge le prestige
adrar - nearest hotel: 

,City,Country,Lat,Lon,Humidity,Hotel Name
17,cape town,ZA,-33.9258,18.4232,82,Townhouse Hotel
26,matamoros,MX,25.5333,-103.2500,43,No hotel found
35,sao filipe,CV,14.8961,-24.4956,76,Bela Vista
38,nishihara,JP,26.1842,127.7558,64,ユインチホテル南城
40,mar del plata,AR,-38.0023,-57.5575,78,Nuevo Ostende
...,...,...,...,...,...,...
485,puerto madryn,AR,-42.7692,-65.0385,61,Hotel Gran Palace
504,porbandar,IN,21.6422,69.6093,52,Toran Tourist Bungalow
511,yol,IN,32.1667,76.2000,22,Hotel Gopal
519,san luis,AR,-33.2950,-66.3356,51,Hotel Dos Venados


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [127]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "Hotel Name",
    frame_width = 500,
    frame_height = 300,
    hover_cols = ['Hotel Name', 'Country']
    )

# Display the map
hvplot.save(map_plot_2, "../output_data/map_plot_2.html")
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (Hotel Name,Country)